In [5]:
n=3

In [44]:
G = SymmetricGroup(n)

In [7]:
SGA = G.algebra(QQ)

In [8]:
A = SGA.dft(); A

[   1    1    1    1    1    1]
[   1 -1/2 -1/2  1/2  1/2   -1]
[   0 -3/4  3/4  3/4 -3/4    0]
[   0    1   -1    1   -1    0]
[   1 -1/2 -1/2 -1/2 -1/2    1]
[   1    1    1   -1   -1   -1]

In [93]:
#is the matrix unitary?
#A*A^T, the Gram matrix, is diagonal, but not the identity. so A is not unitary.

In [9]:
print((A*A.transpose()).str())

[  6   0   0   0   0   0]
[  0   3   0   0   0   0]
[  0   0 9/4   0   0   0]
[  0   0   0   4   0   0]
[  0   0   0   0   3   0]
[  0   0   0   0   0   6]


In [1]:
#define a new DFT which is unitary
#NOTE: in Beals' ['97] he normalizes by \sqrt{d_\lambda/n!}
#but also notes that a basis change is an equivalence relation on rep'ns
#and each equivalence class contains a unitary representatione
#if each rep'n \rho \in \hat{G} is unitary, then the transformation is unitary
#these representations are not unitary
#to make them unitary, use Weyl's unitary trick: 
#define a new inner product <x,y>_\rho = (1/|G|)\sum_{g \in G}<\rho(g)x,\rho(g)y>
#compute an orthonormal basis using this inner product

In [18]:
#define the Fourier coefficient at the representation specht_module
#which is the Specht module corresponding to partition
def hat(f,partition):
    n = sum(partition)
    G = SymmetricGroup(n)
    specht_module = G.algebra(QQ).specht_module(partition)
    d_part = specht_module.dimension()
    return sqrt(d_part/factorial(n))*sum(f(g)*specht_module.representation_matrix(g) for g in G)

In [11]:
#define the delta function delta_s(t) = {1 if s == t, 0 otherwise}
def delta(s):
    return lambda t: 1 if t == s else 0

In [12]:
#for each basis element g \in G
#compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
#make the Fourier coefficients into a list via .list()
#flatten the list of lists to get a vector \hat{g}
#put the vectors into a list and take the transpose to get the DFT
from sage.misc.flatten import flatten
def unitary_dft(n):
    return matrix([flatten([hat(delta(g),partition).list() for partition in Partitions(n)]) for g in G]).transpose()

In [41]:
#define the G-averaged inner product given a representation
#<x,y>_\rho = (1/|G|)\sum_{g \in G}<\rho(g)x,\rho(g)y>
def inv_inner_product(partition,x,y):
    n = sum(partition)
    G = SymmetricGroup(n)
    rho = G.algebra(QQ).specht_module(partition).representation_matrix
    return sum((rho(g)*vector(x)).inner_product(rho(g)*vector(y)) for g in G)

In [104]:
#define the projection of the vector v oto vector u
def proj(u,v):
    return (vector(v).inner_product(vector(u))/vector(u).inner_product(vector(u)))*vector(u)

In [108]:
#define Gram-Schmidt orthonormalization process that works in the symbolic ring
#assume A = [v_1, v_2, ... , v_k] and each v_j is a column vector
def gram_schmidt_SR(A):
    v = A.transpose()
    u = []
    e = []
    for k in range(A.ncols()):
        u_k = v[k] - sum(proj(u[j],v[k]) for j in range(k))
        u.append(u_k)
        e.append(u_k/u_k.norm())
    return u, e

In [109]:
gram_schmidt_SR(A)

([(sqrt(2), 1), (-1/3*sqrt(2)*(2*sqrt(2) + 3) + 2, -2/3*sqrt(2) + 2)],
 [(1/3*sqrt(3)*sqrt(2), 1/3*sqrt(3)),
  (-(sqrt(2)*(2*sqrt(2) + 3) - 6)/sqrt((sqrt(2)*(2*sqrt(2) + 3) - 6)^2 + 4*(sqrt(2) - 3)^2), -2*(sqrt(2) - 3)/sqrt((sqrt(2)*(2*sqrt(2) + 3) - 6)^2 + 4*(sqrt(2) - 3)^2))])

In [110]:
#define the inner product matrix A = [<x_i,x_j>] for some ordered basis of V_\rho
#perform Gram-Schmidt orthogonalization
#ISSUE: the output is not normalized
partition = Partitions(n)[1]
specht_module = G.algebra(QQ).specht_module(partition)
d_part = specht_module.dimension()
e = lambda i: [1 if i==j else 0 for j in range(d_part)]
A = matrix([[inv_inner_product(partition,e(i),e(j)) for j in range(d_part)] for i in range(d_part)])
GS, M = A.gram_schmidt(orthonormal=True)
print(M*GS - A)
try:
    assert M*GS == A
except AssertionError:
    print("Gram-Schmidt orthogonalization failed.")
#note the rows of 

TypeError: QR decomposition unable to compute square roots in Rational Field

In [17]:
inv_inner_product(Partitions(n)[1],vector([1,2]),vector([1,2]))

[ 0 12]
[ 0 15]

In [106]:
unitary_dft(3)*unitary_dft(3).transpose()

[   1    0    0    0    0    0]
[   0  4/3 -2/3  2/3 -1/3    0]
[   0 -2/3  4/3 -1/3  2/3    0]
[   0  2/3 -1/3  4/3 -2/3    0]
[   0 -1/3  2/3 -2/3  4/3    0]
[   0    0    0    0    0    1]

In [58]:
#what are the eigenvalues?
#n=3: two real, two complex
#n=4: all complex
#the magnitude is not 1, they're closely grouped around 2 or 3

In [92]:
eigs = A.eigenvalues(); eigs

[-1.927382440195288?, 2.121715010353017?, -1.174423554578382? - 1.623003052198150?*I, -1.174423554578382? + 1.623003052198150?*I, 1.452257269499517? - 1.086817272076326?*I, 1.452257269499517? + 1.086817272076326?*I]

In [93]:
[abs(eig) for eig in eigs]

[1.927382440195288?,
 2.121715010353017?,
 2.003349593304431?,
 2.003349593304431?,
 1.813897174510622?,
 1.813897174510622?]

In [121]:
#note that the singluar values are the square roots of the diagonal entries of the Gram matrix
print(SymmetricGroup(n).algebra(CDF).dft().SVD()[1].numpy().diagonal())
print(sqrt((A*A.transpose()).numpy().diagonal()))

[2.44948974+0.j 2.44948974+0.j 2.        +0.j 1.73205081+0.j
 1.73205081+0.j 1.5       +0.j]
[2.44948974 1.73205081 1.5        2.         1.73205081 2.44948974]
